In [129]:
import requests
import pandas as pd

### Retrieve the Bitcoin BIP-39 Word List word list, store as a dataframe, and convert it to CSV format

In [130]:
# URL of the BIP-39 Word List
url = 'https://raw.githubusercontent.com/bitcoin/bips/master/bip-0039/english.txt'
filename = 'bip39_word_list'

# Retrieve the word list
response = requests.get(url)
words = response.text.splitlines()

# Convert words to a DataFrame
df = pd.DataFrame({'word': words})
df['11-bit binary'] = df.index.to_series().apply(lambda x: format(x, '011b'))  # Convert word index to 11-bit binary

# Save DataFrame to CSV 
df[['word', '11-bit binary']].to_csv(f'{filename}.csv', index=False, header=False)
print(f'Bitcoin word list has been saved to {filename}.csv')


Bitcoin word list has been saved to bip39_word_list.csv


Now have the word list saved as a dataframe.

In [131]:
df.head()

,word,11-bit binary
0,abandon,00000000000
1,ability,00000000001
2,able,00000000010
3,about,00000000011
4,above,00000000100


We can retreive the binary of a given row like this.

In [132]:
# Retrieve the binary of the first row
binary_of_first_row = df.iloc[1]['11-bit binary']

## Right Rotation Explanation

Right rotation is a bitwise operation that shifts all the bits of a binary number to the right by a specified number of positions. The bits that are shifted out from the right end are reintroduced at the left end.

### Visual Example

Let's take a BIP-39 word and represent it as a binary number. For simplicity, we'll use a short binary example.

Suppose we have a binary number: `10110011`

If we right rotate this binary number by 3 bits, the operation would look like this:

```
Original:  10110011
Rotate by 3: 01110110
```

Here's how it works step-by-step:
1. The last 3 bits `011` are taken out.
2. The remaining bits `10110` are shifted to the right by 3 positions.
3. The bits `011` that were taken out are placed at the beginning.

### BIP-39 Word Example

Each BIP-39 word corresponds to an 11-bit binary number. Let's take an example word and its binary representation:

- Example BIP-39 word: "airport"
- Binary representation: `00000101100`

If we right rotate this binary number by 3 bits, the operation would look like this:

```
Original:  00000101100
Rotate by 3: 10000000101
```

Here's how it works step-by-step:
1. The last 3 bits `100` are taken out.
2. The remaining bits `000001011` are shifted to the right by 3 positions.
3. The bits `100` that were taken out are placed at the beginning.

By performing this right rotation, we get a new binary number `10000000101` which corresponds to the word `level`, a different BIP-39 word.

This operation can be useful in various cryptographic applications where bitwise manipulations are required.

#### Splicing Review
to do the right rotation operation, we need to review splicing

In [133]:
binary_str = "The Original String"
bits = 2
length = len(binary_str)

print("Original:", binary_str)
print("Bit Number:", bits)

print(f'\nFirst {bits} bits: {binary_str[:bits] + "-" * (length - bits)}')
print(f'Last {length - bits} bits: {"-" * (bits) + binary_str[bits:]}')


print(f'\nFirst {length - bits} bits: {binary_str[:-bits:] + "-" * (bits)}')
print(f'Last {bits} bits: {"-" * (bits) + binary_str[-bits:]}')

print(f"Performaing a right rotation of {bits}:")
print("Original:", binary_str)


Original: The Original String
Bit Number: 2

First 2 bits: Th-----------------
Last 17 bits: --e Original String

First 17 bits: The Original Stri--
Last 2 bits: --ng
Performaing a right rotation of 2:
Original: The Original String


**Now that we know splicing, this is how we do a right rotation.**|

In [134]:
binary_str = "00000111111"
bits = 3
length = len(binary_str)

print(f"Original String: {binary_str}")
# to do the right rotation, we need 
# the first part of the string
first_part_of_string = binary_str[:-bits]
print(f"First Part of String: {first_part_of_string + '-' * (bits)}")

# the last part of the string
last_part_of_string = binary_str[-bits:]
print(f"Last Part of String: {'-' * (len(first_part_of_string)) + last_part_of_string}")

# and reverse the order
rotated_binary_string = last_part_of_string + first_part_of_string
print(f"Right Rotated String: {rotated_binary_string}")


Original String: 00000111111
First Part of String: 00000111---
Last Part of String: --------111
Right Rotated String: 11100000111


`right_rotate()` function

In [135]:
# Perform a right rotation
def right_rotate(binary_str, n):
    
    # This line rotates the string `binary_str` to the right by `n` positions.
    # It takes the last `n` characters of `binary_str` and moves them to the front,
    # while the rest of the string is shifted to the right.
    return binary_str[-n:] + binary_str[:-n]

binary_str = "00000111111"
bits = 3
length = len(binary_str)

# Example: Rotate by 3 bits
rotated_binary = right_rotate(binary_str, bits)
rotated_binary

'11100000111'

In [136]:
# Add a new column with the right rotated binary
bits_to_rotate = 1

df[f'rotated_{bits_to_rotate}_bits'] = df['11-bit binary'].apply(lambda x: right_rotate(x, bits_to_rotate))
df.head()

,word,11-bit binary,rotated_1_bits
0,abandon,00000000000,00000000000
1,ability,00000000001,10000000000
2,able,00000000010,00000000001
3,about,00000000011,10000000001
4,above,00000000100,00000000010


In [137]:
# Create a mapping from 11-bit binary to word
binary_to_word = df.set_index('11-bit binary')['word'].to_dict()

# Use the mapping to fill in the new column 'rotated_word'
df[f'word_rotated_{bits_to_rotate}_bits'] = df['rotated_1_bits'].map(binary_to_word)
df.head()

,word,11-bit binary,rotated_1_bits,word_rotated_1_bits
0,abandon,00000000000,00000000000,abandon
1,ability,00000000001,10000000000,length
2,able,00000000010,00000000001,ability
3,about,00000000011,10000000001,lens
4,above,00000000100,00000000010,able


In [145]:
import bitcoin

# Function to convert a list of BIP-39 words to their corresponding 11-bit binary representation
def words_to_binary(words_list, word_to_binary_map):
    return [word_to_binary_map[word] for word in words_list]

# Function to convert binary string to hexadecimal
def binary_to_hex(binary_str):
    return hex(int(binary_str, 2))[2:]

# Function to generate a Bitcoin address from a 12-word key
def generate_bitcoin_address(words_list, word_to_binary_map):
    # Convert words to binary
    binary_list = words_to_binary(words_list, word_to_binary_map)
    
    # Concatenate binary representations
    concatenated_binary = ''.join(binary_list)
    
    # Convert concatenated binary to hexadecimal
    hex_str = binary_to_hex(concatenated_binary)
    
    # Generate Bitcoin address from hexadecimal string
    private_key = bitcoin.decode_privkey(hex_str, 'hex')
    public_key = bitcoin.privkey_to_pubkey(private_key)
    address = bitcoin.pubkey_to_address(public_key)
    
    return address

# Example usage
original_key = ['abandon', 'ability', 'able', 'about', 'above', 'absent', 'absorb', 'abstract', 'absurd', 'abuse', 'access', 'accident']
word_to_binary_map = {v: k for k, v in binary_to_word.items()}  # Create a mapping from word to 11-bit binary

# Generate Bitcoin address
bitcoin_address = generate_bitcoin_address(original_key, word_to_binary_map)
print("Bitcoin Address:", bitcoin_address)
# Generate private key for the original key
original_private_key = bitcoin.decode_privkey(binary_to_hex(''.join(words_to_binary(original_key, word_to_binary_map))), 'hex')
print("Original Private Key:", original_private_key)

# Generate private key for the decoy key
decoy_private_key = bitcoin.decode_privkey(binary_to_hex(''.join(words_to_binary(decoy_key, word_to_binary_map))), 'hex')
print("Decoy Private Key:", decoy_private_key)

Bitcoin Address: 1MznVNecp6mH3uCWJaZWBRsvnYCt7VCgw6
Original Private Key: 1299342794293796661554111911579659
Decoy Private Key: 1329228946832728990915939017622039557


In [146]:
def validate_private_key(private_key):
    try:
        # Convert the private key to a public key
        public_key = bitcoin.privkey_to_pubkey(private_key)
        
        # Convert the public key to a Bitcoin address
        address = bitcoin.pubkey_to_address(public_key)
        
        # If the address is generated successfully, the private key is valid
        return True
    except Exception as e:
        return False

# Validate the original private key
is_original_key_valid = validate_private_key(original_private_key)
print("Is Original Private Key Valid?", is_original_key_valid)

# Validate the decoy private key
is_decoy_key_valid = validate_private_key(decoy_private_key)
print("Is Decoy Private Key Valid?", is_decoy_key_valid)

Is Original Private Key Valid? True
Is Decoy Private Key Valid? True


Retrieve on word